# Machine Learning - Hosting Models with KServe

## Installing KServe

- Install Kind
- Install Kubectl
- Create a cluster with kind


In [1]:
# Create a Cluster
!kind create cluster


Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.27.3) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [2]:
# configure the cluster
!kubectl cluster-info --context kind-kind

Kubernetes control plane is running at https://127.0.0.1:42243
CoreDNS is running at https://127.0.0.1:42243/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [3]:
# prepare the cluster kind-kind
!kubectl config use-context kind-kind


Switched to context "kind-kind".


In [4]:
# Download the kserve script which should install
# the kserve layers and kserve itself

!wget "https://raw.githubusercontent.com/kserve/kserve/release-0.11/hack/quick_install.sh"


--2023-12-04 12:24:50--  https://raw.githubusercontent.com/kserve/kserve/release-0.11/hack/quick_install.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4102 (4.0K) [text/plain]
Saving to: ‘quick_install.sh’

quick_install.sh    100%[===================>]   4.01K  --.-KB/s    in 0.006s  

2023-12-04 12:24:51 (676 KB/s) - ‘quick_install.sh’ saved [4102/4102]



In [5]:
# install the script
!bash quick_install.sh


Flag --short has been deprecated, and will be removed in the future. The --short output will become the default.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   102  100   102    0     0    212      0 --:--:-- --:--:-- --:--:--   212
100  4899  100  4899    0     0   6766      0 --:--:-- --:--:-- --:--:--  6766


Istio 1.17.2 Download Complete!

Istio has been successfully downloaded into the istio-1.17.2 folder on your system.

Next Steps:
See https://istio.io/latest/docs/setup/install/ to add Istio to your Kubernetes cluster.

To configure the istioctl client tool for your workstation,
add the /home/ozkary/workspace/ml-engineering/11-kserve/istio-1.17.2/bin directory to your environment path variable with:
	 export PATH="$PATH:/home/ozkary/workspace/ml-engineering/11-kserve/istio-1.17.2/bin"

Begin the Istio pre-installation check by running:
	 istioctl x precheck 

Nee

In [8]:
# check the nodes in the name space
!kubectl get pods -n kserve

NAME                                         READY   STATUS    RESTARTS   AGE
kserve-controller-manager-78c666cf85-4gz8p   2/2     Running   0          3h32m


## Create the inference service

In [9]:
# Create the test namespace
!kubectl create namespace kserve-test


namespace/kserve-test created


In [14]:
# Create the deployment iris_example.yaml and install it
!kubectl apply -f iris_example.yaml -n kserve-test

inferenceservice.serving.kserve.io/sklearn-iris unchanged


In [1]:
# check the status
!kubectl get inferenceservices sklearn-iris -n kserve-test


NAME           URL                                           READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION            AGE
sklearn-iris   http://sklearn-iris.kserve-test.example.com   True           100                              sklearn-iris-predictor-00001   24h


In [2]:
# determine the ingress IP and port
!kubectl get svc istio-ingressgateway -n istio-system


NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)                                      AGE
istio-ingressgateway   LoadBalancer   10.96.33.146   <pending>     15021:31200/TCP,80:31407/TCP,443:30775/TCP   28h


In [3]:
# create a port forward. This operation runs in the background
# !kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80

Forwarding from 127.0.0.1:8080 -> 8080
Forwarding from [::1]:8080 -> 8080
^C


In [4]:
# set the environment variables
!set INGRESS_HOST = localhost
!set INGRESS_PORT = 8080


In [10]:
# make a prediction using this sample json

payload = { "instances": [
        [6.8,  2.8,  4.8,  1.4],
        [6.0,  3.4,  4.5,  1.6]
        ]}

#!SERVICE_HOSTNAME=$(kubectl get inferenceservice sklearn-iris -n kserve-test -o jsonpath='{.status.url}' | cut -d "/" -f 3)
#!curl -v -H "Host: ${SERVICE_HOSTNAME}" -H "Content-Type: application/json" "http://${INGRESS_HOST}:${INGRESS_PORT}/v1/models/sklearn-iris:predict" -d @./iris-input.json

# use requests to make a prediction using the payload json
import requests
import json

headers = {"Host": "sklearn-iris.kserve-test.example.com", "Content-Type": "application/json"}
url = "http://localhost:8080/v1/models/sklearn-iris:predict"
response = requests.post(url, json=payload, headers=headers)
print(response.text)

# You should see two predictions returned (i.e. {"predictions": [1, 1]}). 
# Both sets of data points sent for inference correspond to the flower with index 1. 
# In this case, the model predicts that both flowers are "Iris Versicolour".

{"predictions":[1,1]}


In [11]:
!kubectl logs load-test8b58n-rgfxr -n kserve-test


Error from server (NotFound): pods "load-test8b58n-rgfxr" not found
